## Library

In [ ]:
from dotenv import load_dotenv

import os
import torch
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, Field
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.vectorstores import FAISS

load_dotenv()

In [ ]:
seed = 1

## Data

In [ ]:
train_df = pd.read_csv("train_형사.csv")
val_df = pd.read_csv("validation_형사.csv")
test_df = pd.read_csv("test_형사.csv")

combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

datasets = {}

test_size = 1000
train_data, test_data = train_test_split(
    combined_df,
    test_size=test_size,
    stratify=combined_df["answer"],
    random_state=seed,
)

val_size = 100
train_data, validation_data = train_test_split(
    train_data,
    test_size=val_size,
    stratify=train_data["answer"],
    random_state=seed,
)

datasets[seed] = {
    "train_data": train_data,
    "test_data": test_data,
    "validation_data": validation_data,
}

train = datasets[seed]["train_data"]
validation = datasets[seed]["validation_data"]
test = datasets[seed]["test_data"]

## Classifier

In [ ]:
class DocumentsClassification(BaseModel):
    prediction: str = Field(description="Classification of the document: True, False")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm_grader = llm.with_structured_output(DocumentsClassification)

## Prompt & Chain

In [ ]:
system_message = """Classify the query into one of : True / False\n
                    "Return in JSON: {{\"prediction\": \"category\"}}
"""
classification_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "Input document: {question}"),
    ]
)

classification_grader = classification_prompt | structured_llm_grader

## Prediction

In [ ]:
results = []
    
for idx, row in tqdm(
    test.iterrows(), total=len(test), desc="Processing"
):
    question = row["question"]
    answer = row["answer"]

    prediction = classification_grader.invoke({"question": question})

    results.append(
        {
            "question": question,
            "answer": answer,
            "prediction": prediction.prediction,
        }
    )

df_results = pd.DataFrame(results)

In [ ]:
y_true = df_results["answer"].map({True: 1, False: 0})
y_pred = df_results["prediction"].map({"True": 1, "False": 0})

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1_final = f1_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average="macro")
f1_weighted = f1_score(y_true, y_pred, average="weighted")

conf_matrix = confusion_matrix(y_true, y_pred)

print("\n===== Classification Performance Results =====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_final:.4f}")
print(f"F1-score (Macro): {f1_macro:.4f}")
print(f"F1-score (Weighted): {f1_weighted:.4f}")

print("\n===== Classification Confusion Matrix =====")
print(conf_matrix)

print("\n===== Detailed Classification Report =====")
print(classification_report(y_true, y_pred))